In [58]:
import threading
import cv2
import time

In [ ]:
class AsyncVideoCapture:
    def __init__(self, source=0):
        self.source = source
        self.capture = cv2.VideoCapture(self.source)
        self.is_frame_grabbed, self.latest_frame = self.capture.read()
        self.is_capturing = False
        self.lock = threading.Lock()

    def start_capturing(self):
        if self.is_capturing:
            print('[!] Asynchronous video capturing has already been started.')
            return

        self.is_capturing = True
        self.capture_thread = threading.Thread(target=self._update_frames)
        self.capture_thread.start()

    def _update_frames(self):
        while self.is_capturing:
            is_frame_grabbed, frame = self.capture.read()
            with self.lock:
                self.is_frame_grabbed = is_frame_grabbed
                self.latest_frame = frame

    def read_latest_frame(self):
        with self.lock:
            frame = self.latest_frame.copy() if self.latest_frame is not None else None
            is_frame_grabbed = self.is_frame_grabbed
        return is_frame_grabbed, frame

    def stop_capturing(self):
        self.is_capturing = False
        self.capture_thread.join()

    def release_resources(self):
        self.capture.release()

    def __del__(self):
        self.release_resources()

In [ ]:
def asynchronous_video_capture(source=0):
    try:
        video_capture = AsyncVideoCapture(source=source)
        video_capture.start_capturing()

        start_time = time.time()
        frame_number = 0

        while True:
            _, frame = video_capture.read_latest_frame()
            if frame is None:
                print("Source ended")
                break

            stop_time = time.time()
            total_time = stop_time - start_time
            frame_number = frame_number + 1
            sync_fps = frame_number / total_time

            cv2.putText(frame, "Asynchronous Video Capture", (5, 60), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255), 2)
            cv2.putText(frame, str(round(sync_fps, 2)) + " FPS", (5, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2)
            cv2.namedWindow("Asynchronous Video Capture", cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)
            cv2.imshow('Asynchronous Video Capture', frame)

            k = cv2.waitKey(1) & 0xFF
            if k == 27:
                break


    except Exception as e:
        print(f'[!] An error occurred: {e}')

    finally:
        video_capture.stop_capturing()
        cv2.destroyAllWindows()

In [59]:
asynchronous_video_capture(source=2)
